### Description

This notebook demonstrates performing multiple queries again gpt-4o-mini with the underlying data being UDS Package Bookstack failures due to missing UDS Exemption resources

In [13]:
# get UDS Core docs

import os
from pathlib import Path
from openai import OpenAI

def read_file(file_path):
    """Read the contents of a file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def process_markdown_files(folder_path):
    """Process all .md and .mdx files in the given folder and its subfolders,
    excluding 'adrs' folders and '_index.md' files."""
    combined_content = ""
    for root, dirs, files in os.walk(folder_path):
        # Exclude 'adrs' folders
        dirs[:] = [d for d in dirs if d != "adrs"]
        
        for file in files:
            if file.endswith(('.md', '.mdx')) and file != "_index.md":
                file_path = Path(root) / file
                content = read_file(file_path)
                combined_content += f"\n\nContent of {file_path}:\n{content}"
    return combined_content

log_file = "bookstack-no-exemption-error.txt"
event_file = "bookstack-no-exemption-events.txt"
pepr_logs_file = "bookstack-pepr-no-exemption.txt"

docs_folder = "../uds/uds-core/docs"
operator_docs_folder = "../uds/uds-core/src/pepr"
model = "gpt-4o-mini"
    
logs = read_file(log_file)
events = read_file(event_file)

uds_docs_content = process_markdown_files(docs_folder)
operator_docs_content = process_markdown_files(operator_docs_folder)

query = """
    What's the state of the applications running in the "bookstack" namespace
"""

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

system_prompt = f"""
    You are an expert in Kubernetes cluster analysis. 
    You are analyzing a K8s cluster running UDS, docs for UDS are as follows:
    
    {uds_docs_content}
    
    This UDS cluster also includes a UDS K8s operator, docs the operator are as follows:%%!
    
    {operator_docs_content}

    These operator docs describe an Exemption resource. This is often used to diagnose security issues similar to Kyverno
    
    Given this knowledge about UDS and UDS K8s Operator, analyze the provided logs and events to answer questions about applications running in the cluster.
"""

user_prompt = f"""
    Here are the recent logs from the "bookstack" namespace:
    
    {logs}
    
    Here are the recent events from the "bookstack" namespace:
    
    {events}

    Here are the recent logs from the UDS K8s Operator, these could provide valuable information related to the logs and events:%%!

    {pepr_logs_file}
    
    Based on all this information, please answer the following question: {query}
    
    Be brief in your response, but if something is broken, suggest how we can fix it
"""

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)

answer = response.choices[0].message.content
print(answer)

The state of the applications running in the "bookstack" namespace indicates that there are issues specifically with the "bookstack" application pods.

1. **Pod Initialization Failures**: 
   - The pods `bookstack-bookstack-bc6cb49db-khz52` and `bookstack-bookstack-bc6cb49db-dgpv5` are in a `PodInitializing` state, and the container "bookstack" is waiting to start. This is primarily due to the error messages "chown: /mnt/uploads: Operation not permitted", indicating permission issues when trying to set ownership on that directory.

2. **Volume Initialization Failures**: 
   - The `volume-init` container in both `bookstack` pod replicas has failed to restart, leading to a `BackOff` state. This suggests that it is unable to complete its intended operation, potentially due to the aforementioned permission errors.

3. **Database Pod Running**: 
   - The `bookstack-mysql` pods seem to have initialized correctly and are running without issue, as indicated by the successful creation, starting

In [17]:
# test knowledge about the docs I gave the LLM

import os
from pathlib import Path
from openai import OpenAI

def read_file(file_path):
    """Read the contents of a file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def process_markdown_files(folder_path):
    """Process all .md and .mdx files in the given folder and its subfolders,
    excluding 'adrs' folders and '_index.md' files."""
    combined_content = ""
    for root, dirs, files in os.walk(folder_path):
        # Exclude 'adrs' folders
        dirs[:] = [d for d in dirs if d != "adrs"]
        
        for file in files:
            if file.endswith(('.md', '.mdx')) and file != "_index.md":
                file_path = Path(root) / file
                content = read_file(file_path)
                combined_content += f"\n\nContent of {file_path}:\n{content}"
    return combined_content

docs_folder = "../uds/uds-core/docs"
operator_docs_folder = "../uds/uds-core/src/pepr"
model = "gpt-4o-mini"

uds_docs_content = process_markdown_files(docs_folder)
operator_docs_content = process_markdown_files(operator_docs_folder)

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

system_prompt = f"""
    You are an expert in Kubernetes cluster analysis. 
    You are analyzing a K8s cluster running UDS (Unicorn Delivery Service), docs for UDS are as follows:
    
    {uds_docs_content}
    
    This UDS cluster also includes a UDS K8s operator, docs the operator are as follows:%%!
    
    {operator_docs_content}

    Please pay close attention to the ideas of a UDS Exemption and UDS Package resource!
"""

user_prompt = f"""
    Tell me what you know of UDS and the UDS K8s Operator
"""

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)

uds_docs_answer = response.choices[0].message.content
print(uds_docs_answer)

**UDS (Unicorn Delivery Service)** is a foundational set of applications designed to support secure and efficient microservice communication, monitoring, logging, security, compliance, and data protection within a Kubernetes environment. UDS Core provides a collection of essential applications and capabilities that help ensure mission-critical applications operate seamlessly and meet stringent security and performance standards. The key applications included in UDS Core are:

1. **Service Mesh**: Powered by **Istio**, it manages traffic, load balancing, security, and observability across microservices.
2. **Monitoring**: Leveraging the **Prometheus Stack** for metrics and health insights, along with **Grafana** for visualization.
3. **Logging**: Integrated with **Loki** for log aggregation and **Promtail** for log collection.
4. **Security and Compliance**: Includes **NeuVector** for container security and **Pepr** for policy enforcement.
5. **Identity and Access Management**: Utilizes

In [27]:
# all together now! (multiple queries)

import os
from pathlib import Path
from openai import OpenAI

def read_file(file_path):
    """Read the contents of a file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def process_markdown_files(folder_path):
    """Process all .md and .mdx files in the given folder and its subfolders,
    excluding 'adrs' folders and '_index.md' files."""
    combined_content = ""
    for root, dirs, files in os.walk(folder_path):
        # Exclude 'adrs' folders
        dirs[:] = [d for d in dirs if d != "adrs"]
        
        for file in files:
            if file.endswith(('.md', '.mdx')) and file != "_index.md":
                file_path = Path(root) / file
                content = read_file(file_path)
                combined_content += f"\n\nContent of {file_path}:\n{content}"
    return combined_content

def chat_with_gpt(messages, model="gpt-4o-mini"):
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

# File paths

# missing Exemption
# log_file = "bookstack-no-exemption-error.txt"
# event_file = "bookstack-no-exemption-events.txt"
# pepr_logs_file = "bookstack-pepr-no-exemption.txt"

# missing db
log_file = "bookstart-err-db-logs.txt"
event_file = "bookstack-err-db-events.txt"
pepr_logs_file = "bookstack-pepr-missing-db.txt"

docs_folder = "../uds/uds-core/docs"
operator_docs_folder = "../uds/uds-core/src/pepr"

# Process documentation
uds_docs_content = process_markdown_files(docs_folder)
operator_docs_content = process_markdown_files(operator_docs_folder)

# Initial system prompt with documentation
system_prompt = f"""
You are an expert in Kubernetes cluster analysis. 
You are analyzing a K8s cluster running UDS (Unicorn Delivery Service), docs for UDS are as follows:

{uds_docs_content}

This UDS cluster also includes a UDS K8s operator built using Pepr, docs for the operator are as follows:

{operator_docs_content}

Please pay close attention to the ideas of a UDS Exemption and UDS Package resource!
"""

# First interaction: Understand UDS and UDS K8s Operator
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Tell me what you know of UDS and the UDS K8s Operator"}
]

uds_docs_answer = chat_with_gpt(messages)
# print("UDS and UDS K8s Operator Understanding:")
# print(uds_docs_answer)

# Second interaction: Analyze logs and events
logs = read_file(log_file)
events = read_file(event_file)
pepr_logs = read_file(pepr_logs_file)

analysis_prompt = f"""
Given your understanding of UDS and the UDS K8s Operator, analyze the provided logs and events to answer questions about applications running in the cluster.

Here are the recent logs from the "bookstack" namespace:

{logs}

Here are the recent events from the "bookstack" namespace:

{events}

Here are the recent logs from the UDS K8s Operator, these could provide valuable information related to the logs and events:

{pepr_logs}

Based on all this information, please answer the following question: What's the state of the applications running in the "bookstack" namespace?

Be brief in your response, don't forget about UDS Exemptions and Packages, and if something is broken, suggest a couple of actions
"""

messages.append({"role": "assistant", "content": uds_docs_answer})
messages.append({"role": "user", "content": analysis_prompt})

analysis_answer = chat_with_gpt(messages)
print("\nAnalysis of 'bookstack' namespace:")
print(analysis_answer)



Analysis of 'bookstack' namespace:
### State of Applications in the "bookstack" Namespace

The application in the "bookstack" namespace, primarily the **Bookstack** application, is in a partially healthy state:

1. **Pod Creation and Scheduling:**
   - A pod (`bookstack-bookstack-bc6cb49db-7p4km`) was successfully scheduled and created. Initial events indicate that it pulled the required image and started a container (`volume-init`).

2. **Readiness and Initialization:**
   - The logs indicate the readiness probe succeeded, and the Envoy proxy is ready, suggesting that the application layer is prepared to handle requests.

3. **Database Connectivity Issue:**
   - However, there are significant issues with connecting to the **MySQL database** (`bookstack-mysql`). Several log entries show repeated failures in DNS resolution for `bookstack-mysql`, indicating that the application cannot connect to its database, which is critical for its operation.

4. **Container Restart and Back-off:**
 